In [ ]:
import numpy as np
from scipy.sparse import csr_matrix

In [ ]:
def lu(A):
    assert len(A.shape) == 2, "only 2d matrices are supported"
    assert A.shape[0] == A.shape[1], "only nxn matrices are supported"

    n = A.shape[0]
    L, U = np.zeros_like(A), np.zeros_like(A)

    for i in range(n):
        L[i, i] = 1

        for k in range(i, n):
            U[i, k] = A[i, k] - np.sum(L[i] * U[:, k])

        for k in range(i + 1, n):
            L[k, i] = (A[k, i] - np.sum(L[i] * U[:, k])) / U[i, i]

    return csr_matrix(L), csr_matrix(U)


In [ ]:

matrix = csr_matrix([
    [1, 0, 3],
    [0, 3, 1],
    [0, 0, 6]
], shape=(3, 3))
L, U = lu(matrix)
print(L.toarray())
print(U.toarray())
